In [ ]:
# Importation des bases

import functions as fc

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data['emissions'].head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dechets_dang.shape # 72207

In [ ]:
dechets_dang.head(1)